In [ ]:
#Look Ma a Spike


In [ ]:
import math
import mne
from MEG_Tools import MEG
import matplotlib as plt
import numpy as np
plt.use('Qt5Agg')
meg = MEG('case_2225_with_spike_dipoles_sleep_2.mat')
mne_fif = meg.get_mne()

In [ ]:
#First we find the location of a known spike

known_spike_time = meg.spikes[0]['begin']
known_spike_time = int(math.floor(known_spike_time))#round to second below
known_spike_time

In [ ]:
def round_down_to_nearest_epoch(time, epoch_length):
    # Rounds the time down to the nearest multiple of epoch_length
    return int(math.floor(time / epoch_length))


In [ ]:
#Make epochs
length_of_epoch = 1 #number of seconds

sfreq = mne_fif.info['sfreq']
#events = np.array([[i, 0, 1] for i in range(0, mne_fif.n_times - int(sfreq), int(sfreq))], dtype=int)
events = np.array([[i, 0, 1] for i in range(0, mne_fif.n_times - int(length_of_epoch * sfreq), int(length_of_epoch * sfreq))], dtype=int)

#epochs = mne.Epochs(mne_fif, events, event_id={'one_sec': 1}, tmin=0, tmax=1, baseline=None, preload=True)
epochs = mne.Epochs(mne_fif, events, event_id={str(length_of_epoch)+" sec": 1}, tmin=0, tmax=3, baseline=None, preload=True)


In [ ]:
#Let's take a look....
first_spike = epochs[round_down_to_nearest_epoch(known_spike_time,length_of_epoch)]

# Plot the first epoch
first_spike.plot()

In [ ]:
#plot spikes & epochs
epochs.plot(scalings={'mag': 3.0e-11,'eeg': 1.0e-3})  # Manual adjustment

In [ ]:
#plot - Wavelet Spike
frequencies = np.arange(4, 70, 1)
spike_time = round_down_to_nearest_epoch(known_spike_time,length_of_epoch)
power = mne.time_frequency.tfr_morlet(
    epochs[spike_time:spike_time+1], n_cycles=2, return_itc=False, freqs=frequencies, decim=3
)
power.plot(["MEG0242"])

In [ ]:
#plot - Wavelet Non-Spike
frequencies = np.arange(4, 70, 1)
power = mne.time_frequency.tfr_morlet(
    epochs[length_of_epoch*3:length_of_epoch*4], n_cycles=2, return_itc=False, freqs=frequencies, decim=3
)
power.plot(["MEG0242"])

In [ ]:
#plot 
epochs[spike_time-1:spike_time+1].plot_image(picks=["MEG0242"])

In [ ]:
def get_spiked_lists(epochs_,meg_):
    epochlength = round_down_to_nearest_epoch(len(epochs_), length_of_epoch)
  
    spiked_list_ = []
    for spike in meg_.spikes:
        begin = spike['begin']
        rounded_begin = round_down_to_nearest_epoch(begin, length_of_epoch)
        spiked_list_.append(rounded_begin)
    
    # Remove duplicates and sort spiked_list
    spiked_list_ = sorted(list(set(spiked_list_)))
    
    # Creating unspiked_list
    unspiked_list_ = [i for i in range(epochlength) if i not in spiked_list_]
    
    spiked_epochs_ = epochs_[spiked_list_]
    unspiked_epochs_ = epochs_[unspiked_list_]
    return (spiked_list_,unspiked_list_,spiked_epochs_,unspiked_epochs_)



In [ ]:
(spiked_list,unspiked_list,spiked_epochs, unspiked_epochs) = get_spiked_lists(epochs,meg)

In [ ]:
#Plot
#GFP - "Global Field Power"
#GFP is calculated by summing the squared signals from all sensors or channels across the scalp or head. 
#It can be used to analyze and visualize the overall intensity of brain activity, 
#helping researchers identify periods of high neural engagement or specific brain regions that are 
#particularly active during certain tasks or conditions.

spiked_evoked = spiked_epochs.average()
unspiked_evoked = unspiked_epochs.average()

mne.viz.plot_compare_evokeds(
    dict(spiked=spiked_evoked, unspiked=unspiked_evoked),
    legend="upper left",
    show_sensors="upper right",
)

In [ ]:
#Plot
evoked_diff = mne.combine_evoked([spiked_evoked, unspiked_evoked], weights=[1, -1])
evoked_diff.pick(picks="mag").plot_topo(color="r", legend=False)

In [ ]:
#Plot - Hmmmm
spiked_evoked.plot_joint(picks="mag")
spiked_evoked.plot_topomap(times=[0.0, 0.08, 0.1, 0.12, 0.2], ch_type="mag")